## Hyperparameter Tuning in MLP Model

In the below CNN model, The input layer, **number of filters** and the **activation function** for the model is determined using Keras tuner, with the goal of minimum MSE loss.

In [2]:
from numpy import array
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
from kerastuner.tuners import RandomSearch

from math import log
from math import exp
from scipy.stats import boxcox

In [3]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [4]:
data = pd.read_csv("Timeseries_real_data_interview.csv")
t = data.head(16000)
v = data.tail(4000)
# define input sequence
train_seq = t['Value'].values
val_seq = v['Value'].values

tr, tr_lamb = boxcox(train_seq)
vl, vl_lamb = boxcox(val_seq)

In [10]:
# choose a number of time steps
n_steps_in, n_steps_out = 30, 30
# split into samples
X_train, y_train = split_sequence(tr, n_steps_in, n_steps_out)
X_val, y_val = split_sequence(vl, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1

### Hyperparameter model

In [16]:
def build_model(hp):
    
    nodes = hp.Int('nodes', min_value=1, max_value=150, step=10)
    act_func = hp.Choice('Activation Func', ['relu', 'elu', 'sigmoid', 'tanh'])
    
    model = Sequential()
    model.add(Dense(140, activation=act_func, input_dim=n_steps_in))
    model.add(Dropout(0.2))
    model.add(Dense(n_steps_out))

    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=adam, loss='mse', metrics=['mae', keras.metrics.RootMeanSquaredError()])
    return model

In [17]:
tuner = RandomSearch(
    build_model,
    objective='loss',
    seed=1,
    max_trials=16,
    executions_per_trial=2,
    directory='dir',
    project_name='mlp_set1')

In [18]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
Activation Func (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu', 'sigmoid', 'tanh'], 'ordered': False}


In [19]:
tuner.search(X_train, y_train, epochs=20, verbose=1, validation_data=(X_val, y_val), batch_size=64)

Trial 4 Complete [00h 01m 22s]
loss: 10.414721488952637

Best loss So Far: 8.899535179138184
Total elapsed time: 00h 05m 37s
INFO:tensorflow:Oracle triggered exit


<hr>

# Results

The tuning was performed in 2 seperate sets. The first set focused on tuning the number of filters only, followed by the tuning of activation function.

<================ *Results of tuning the number of filters.* =================>

*Search space* : (min_value=50, max_value=150, step=1)

Choice: From the below results, the value of _**126**_ has been chosen for the number of filters in the input layer, since it has loss of 8.208714008331299

In [15]:
tuner.results_summary()

Results summary
Results in dir\mlp_set0
Showing 10 best trials
Objective(name='loss', direction='min')
Trial summary
Hyperparameters:
nodes: 141
Score: 9.689944744110107
Trial summary
Hyperparameters:
nodes: 121
Score: 10.071916103363037
Trial summary
Hyperparameters:
nodes: 91
Score: 11.684062480926514
Trial summary
Hyperparameters:
nodes: 71
Score: 12.655460834503174
Trial summary
Hyperparameters:
nodes: 51
Score: 14.436402797698975
Trial summary
Hyperparameters:
nodes: 41
Score: 17.140339851379395
Trial summary
Hyperparameters:
nodes: 31
Score: 18.94130516052246
Trial summary
Hyperparameters:
nodes: 21
Score: 22.677650451660156


<hr>

<================ *Results of tuning the activation function.* =================>

*Search space* : ['relu', 'elu', 'sigmoid', 'tanh']

Choice: From the below results, the activation function -**tanh**_ has been chosen for the number of filters in the input layer, since it has loss of 6.956228494644165

In [20]:
tuner.results_summary()

Results summary
Results in dir\mlp_set1
Showing 10 best trials
Objective(name='loss', direction='min')
Trial summary
Hyperparameters:
Activation Func: elu
Score: 8.899535179138184
Trial summary
Hyperparameters:
Activation Func: relu
Score: 9.79287576675415
Trial summary
Hyperparameters:
Activation Func: tanh
Score: 10.414721488952637
Trial summary
Hyperparameters:
Activation Func: sigmoid
Score: 10.952197551727295
